In [1]:
import pandas as pd
import numpy as np 
from scipy.stats import pearsonr
from sklearn.metrics import classification_report

In [2]:
from scipy.stats import norm, fisher_exact

In [6]:
for path in ["K562_cov_atac.tsv", "K562_cov_dnase.tsv", "HepG2_cov_atac.tsv", "HepG2_cov_dnase.tsv"]:
    print(path)
    data = pd.read_csv(f"../human_legnet/{path}", sep="\t")
    data['ref_pred'] = data.loc[:, data.columns.str.startswith("pred_ref")].mean(axis=1)
    data['alt_pred'] = data.loc[:, data.columns.str.startswith("pred_alt")].mean(axis=1)
    N =  data.columns.str.startswith("pred_ref").sum()

    var_ref = data.loc[:, data.columns.str.startswith("pred_ref")].var(axis=1).mean()
    var_alt = data.loc[:, data.columns.str.startswith("pred_alt")].var(axis=1).mean()
    var = var_ref + var_alt
    std = np.sqrt(var / N)
    #se = std * 1.64485

    print("All data, Pearson R2", pearsonr(data['score'], data['ref_pred'] - data['alt_pred'])[0])
    
    data['pred_score'] = data['ref_pred'] - data['alt_pred']
    
    data['pred_pval'] = norm.cdf(-abs(data['pred_score']) / std)
    pred_significant =  data['pred_pval'] < 0.05
    
    
    data['pred_pref_allele'] = (data['ref_pred'] - data['alt_pred']).apply(lambda x: "ref" if x > 0 else "alt")
    
    mask = data['fdr_comb_pval'] < 0.05

    print("FDR < 0.05, data, Pearson R2", pearsonr(data[mask]['score'], data[mask]['ref_pred'] - data[mask]['alt_pred'])[0])
    
    data_sig = data[mask].copy()
    crosstab = pd.crosstab(index=data_sig['pref_allele'], columns=data_sig['pred_pref_allele'])
    crosstab = crosstab.loc[['ref', 'alt'], ['ref', 'alt']]
    print("Data without thresholding model predictions")
    print(crosstab)
    print(fisher_exact(crosstab))
    
    
    mask = np.logical_and(data['fdr_comb_pval'] < 0.05, pred_significant)
    data_sig = data[mask].copy()
    print("FDR < 0.05, model pval, data, Pearson R2", pearsonr(data_sig['score'], 
                                                               data_sig['ref_pred'] - data_sig['alt_pred'])[0])
    
    
    crosstab = pd.crosstab(index=data_sig['pref_allele'], columns=data_sig['pred_pref_allele'])
    crosstab = crosstab.loc[['ref', 'alt'], ['ref', 'alt']]
    print("Data with thresholding model predictions")
    print(crosstab)
    print(fisher_exact(crosstab))
    
    final_data =  data[['chr',
      'start',
      'end', 
      'ref',
      'alt', 
      'ref_pred',
      'alt_pred',
      'fdr_comb_pval',
      'pred_pval',
      'score',
      'pred_score',
      'pref_allele',
      'pred_pref_allele']]
    
    name = path.replace(".tsv", "_analysis.tsv")
    #final_data.to_csv(name, sep="\t", index=None)

K562_cov_atac.tsv
All data, Pearson R2 0.09699208465104389
FDR < 0.05, data, Pearson R2 0.2612582119502353
Data without thresholding model predictions
pred_pref_allele  ref  alt
pref_allele               
ref               145   72
alt               101  154
SignificanceResult(statistic=3.0706820682068208, pvalue=4.654851873535244e-09)
FDR < 0.05, model pval, data, Pearson R2 0.33434609309767915
Data with thresholding model predictions
pred_pref_allele  ref  alt
pref_allele               
ref                96   38
alt                47   93
SignificanceResult(statistic=4.998880179171333, pvalue=2.3731087609957486e-10)
K562_cov_dnase.tsv
All data, Pearson R2 0.16271177747975757
FDR < 0.05, data, Pearson R2 0.3487605002191143
Data without thresholding model predictions
pred_pref_allele  ref  alt
pref_allele               
ref               786  399
alt               435  809
SignificanceResult(statistic=3.663607294097312, pvalue=1.4892206224325915e-54)
FDR < 0.05, model pval, data, Pear

In [206]:
np.logical_and(final_data['fdr_comb_pval'] < 0.05, final_data['pred_pval'] < 0.05).sum()

274

In [207]:
 96 + 93

189

In [8]:
eps=1e-10

In [10]:
for path in ["K562_asb.tsv", "HepG2_asb.tsv"]:
    print(path)
    data = pd.read_csv(f"../human_legnet/{path}", sep="\t")
    data['ref_pred'] = data.loc[:, data.columns.str.startswith("pred_ref")].mean(axis=1)
    data['alt_pred'] = data.loc[:, data.columns.str.startswith("pred_alt")].mean(axis=1)
    data['pref_allele'] = data[['fdrp_bh_ref',
           'fdrp_bh_alt']].apply(lambda x: 'ref' if x.fdrp_bh_ref < x.fdrp_bh_alt else "alt", axis=1)
    data['score'] = np.log2(data['min_fdr'] + eps) * (data['pref_allele'].apply(lambda x: -1 if x=='ref' else 1))


    N =  data.columns.str.startswith("pred_ref").sum()

    var_ref = data.loc[:, data.columns.str.startswith("pred_ref")].var(axis=1).mean()
    var_alt = data.loc[:, data.columns.str.startswith("pred_alt")].var(axis=1).mean()
    var = var_ref + var_alt
    std = np.sqrt(var / N)
    #se = std * 1.64485

    print("All data, Pearson R2", pearsonr(data['score'], data['ref_pred'] - data['alt_pred'])[0])
    
    data['pred_score'] = data['ref_pred'] - data['alt_pred']
    
    data['pred_pval'] = norm.cdf(-abs(data['pred_score']) / std)
    pred_significant =  data['pred_pval'] < 0.05
    
    
    data['pred_pref_allele'] = (data['ref_pred'] - data['alt_pred']).apply(lambda x: "ref" if x > 0 else "alt")
    
    mask = data['min_fdr'] < 0.05

    print("FDR < 0.05, data, Pearson R2", pearsonr(data[mask]['score'], data[mask]['ref_pred'] - data[mask]['alt_pred'])[0])
    
    data_sig = data[mask].copy()
    crosstab = pd.crosstab(index=data_sig['pref_allele'], columns=data_sig['pred_pref_allele'])
    crosstab = crosstab.loc[['ref', 'alt'], ['ref', 'alt']]
    print("Data without thresholding model predictions")
    print(crosstab)
    print(fisher_exact(crosstab))
    
    data['fdr_comb_pval'] = data['min_fdr'] 
    data['start'] = data['pos']
    data['end'] = data['pos'] + 1
    
    mask = np.logical_and(data['fdr_comb_pval'] < 0.05, pred_significant)
    data_sig = data[mask].copy()
    print("FDR < 0.05, model pval, data, Pearson R2", pearsonr(data_sig['score'], 
                                                               data_sig['ref_pred'] - data_sig['alt_pred'])[0])
    
    
    crosstab = pd.crosstab(index=data_sig['pref_allele'], columns=data_sig['pred_pref_allele'])
    crosstab = crosstab.loc[['ref', 'alt'], ['ref', 'alt']]
    print("Data with thresholding model predictions")
    print(crosstab)
    print(fisher_exact(crosstab))
    
    final_data =  data[['chr',
      'start',
      'end', 
      'ref',
      'alt', 
      'ref_pred',
      'alt_pred',
      'fdr_comb_pval',
      'pred_pval',
      'score',
      'pred_score',
      'pref_allele',
      'pred_pref_allele']]
    
    name = path.replace(".tsv", "_analysis.tsv")
    final_data.to_csv(name, sep="\t", index=None)

K562_asb.tsv
All data, Pearson R2 0.07088269406962822
FDR < 0.05, data, Pearson R2 0.16694896021607136
Data without thresholding model predictions
pred_pref_allele   ref   alt
pref_allele                 
ref               7212  5560
alt               6590  8106
SignificanceResult(statistic=1.5955194812283708, pvalue=1.9783535510627323e-82)
FDR < 0.05, model pval, data, Pearson R2 0.21331963355095973
Data with thresholding model predictions
pred_pref_allele   ref   alt
pref_allele                 
ref               4290  2868
alt               3311  4628
SignificanceResult(statistic=2.0907991492792504, pvalue=2.2678725750157555e-111)
HepG2_asb.tsv
All data, Pearson R2 0.08018506978382701
FDR < 0.05, data, Pearson R2 0.26180842634035567
Data without thresholding model predictions
pred_pref_allele   ref   alt
pref_allele                 
ref               2959  2086
alt               1868  2667
SignificanceResult(statistic=2.025241438282339, pvalue=1.7651682662523761e-65)
FDR < 0.05, mod

In [214]:
2.2678e-111

,chr,pos,ID,ref,alt,repeat_type,n_peak_calls,n_peak_callers,mean_BAD,mean_SNP_per_segment,...,pred_ref_test6_val3_0_rev,pred_alt_test6_val3_0_forw,pred_alt_test6_val3_0_rev,ref_pred,alt_pred,pref_allele,pred_score,pred_pval,pred_pref_allele,fdr_comb_pval
0,chr1,770502,rs72631875,G,A,SINE,0,0,1.00,1105.0,...,-0.40330,-0.4248,-0.4020,-0.407058,-0.421958,no,0.014899,1.582573e-02,ref,1.000000e+00
1,chr1,888410,rs143626389,G,A,Low_complexity,0,0,1.10,805.9,...,-0.21510,-0.3235,-0.1935,-0.263535,-0.243948,alt,-0.019587,2.365418e-03,alt,2.364817e-03
2,chr1,903551,rs7523690,A,C,LTR,0,0,1.03,1075.7,...,-0.52600,-0.4888,-0.5566,-0.508631,-0.528689,no,0.020058,1.909637e-03,ref,1.000000e+00
3,chr1,905373,rs4970382,T,C,LINE,0,0,1.09,1305.8,...,-0.22730,-0.2622,-0.2125,-0.113151,-0.012569,no,-0.100583,5.555454e-48,alt,8.868125e-02
4,chr1,908025,rs11516185,A,G,NaN,0,0,1.17,1579.0,...,-0.56300,-0.5110,-0.5547,-0.557515,-0.559646,no,0.002131,3.793201e-01,no,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327934,chrX,155179766,rs6567793,A,G,LINE,0,0,1.50,235.0,...,-0.26340,-0.2803,-0.2600,-0.261345,-0.254826,alt,-0.006519,1.735502e-01,no,5.104239e-07
327935,chrX,155613005,rs89278,C,T,NaN,0,0,2.50,166.0,...,0.31180,0.5140,0.4045,0.489073,0.537469,no,-0.048396,1.480480e-12,alt,1.000000e+00
327936,chrX,155662569,rs553678,A,G,NaN,0,0,1.50,235.0,...,-0.20910,-0.2487,-0.2081,-0.212276,-0.203760,no,-0.008516,1.096976e-01,no,1.000000e+00
327937,chrX,155881414,rs28413172,C,T,NaN,0,0,2.25,547.0,...,0.11664,0.0511,0.1456,-0.007651,0.033403,no,-0.041055,1.601459e-09,alt,1.000000e+00


In [151]:
for path in ["K562_asb.tsv", "HepG2_asb.tsv"]:

    data = pd.read_csv(f"../human_legnet/{path}", sep="\t")
    data['ref_pred'] = data.loc[:, data.columns.str.startswith("pred_ref")].mean(axis=1)
    data['alt_pred'] = data.loc[:, data.columns.str.startswith("pred_alt")].mean(axis=1)

    data['pref_allele'] = data[['fdrp_bh_ref',
           'fdrp_bh_alt']].apply(lambda x: 'ref' if x.fdrp_bh_ref < x.fdrp_bh_alt else "alt", axis=1)

    data['score'] = np.log2(data['min_fdr'] + eps) * (data['pref_allele'].apply(lambda x: -1 if x=='ref' else 1))


    var_ref = data.loc[:, data.columns.str.startswith("pred_ref")].var(axis=1).mean()
    var_alt = data.loc[:, data.columns.str.startswith("pred_alt")].var(axis=1).mean()
    var = var_ref + var_alt
    std = np.sqrt(var / N)
    #se = std * 1.64485
    
    data['pred_score'] = data['ref_pred'] - data['alt_pred']
    
    data['pred_pval'] = norm.cdf(-abs(data['pred_score']) / std)


    print("All data, Pearson R2", pearsonr(data['score'], data['ref_pred'] - data['alt_pred'])[0])

    significant = abs(data['ref_pred'] - data['alt_pred']) > se 
    data['pred_pref'] = (data['ref_pred'] - data['alt_pred']).apply(lambda x: "ref" if x > 0 else "alt")
    
    mask = data['min_fdr'] < 0.05
    
    print("FDR < 0.05, data, Pearson R2", pearsonr(data[mask]['score'], data[mask]['ref_pred'] - data[mask]['alt_pred'])[0])
    
    
    data_sig = data[mask].copy()
    crosstab = pd.crosstab(index=data['pref_allele'], columns=data['pred_pref_allele'])
    crosstab = crosstab.loc[['ref', 'alt'], ['ref', 'alt']]
    print("Data without thresholding model predictions")
    print(crosstab)
    
    
    data.loc[~significant, 'pred_pref'] = "no"
    data.loc[data['min_fdr'] > 0.05, 'pref_allele'] = "no"
    crosstab = pd.crosstab(index=data['pref_allele'], columns=data['pred_pref'])
    crosstab = crosstab.loc[['ref', 'alt', 'no'], ['ref', 'alt', 'no']]
    print(crosstab)

All data, Pearson R2 0.07088269406962822
FDR < 0.05, data, Pearson R2 0.16694896021607136
pred_pref      ref    alt      no
pref_allele                      
ref           4290   2868    5614
alt           3311   4628    6757
no           62173  61472  176826
All data, Pearson R2 0.08018506978382701
FDR < 0.05, data, Pearson R2 0.26180842634035567
pred_pref      ref    alt      no
pref_allele                      
ref           1661    887    2497
alt            781   1518    2236
no           51233  50806  174677


In [111]:
eps =1e-10
tp = "single"
data = pd.read_csv("../human_legnet/HepG2_asb.tsv", sep="\t")
data['ref_pred'] = data.loc[:, data.columns.str.startswith("pred_ref")].mean(axis=1)
data['alt_pred'] = data.loc[:, data.columns.str.startswith("pred_alt")].mean(axis=1)
data['ref_pred_std'] = data.loc[:, data.columns.str.startswith("pred_ref")].std(axis=1)
data['alt_pred_std'] = data.loc[:, data.columns.str.startswith("pred_alt")].std(axis=1)
data['pref_allele'] = data[['es_mean_ref',
       'es_mean_alt']].apply(lambda x: 'ref' if x.es_mean_ref > x.es_mean_alt else "alt", axis=1)

data['score'] = np.log2(data['min_fdr'] + eps) * (data['pref_allele'].apply(lambda x: -1 if x=='ref' else 1))

data['pred_std'] =  np.sqrt((data['ref_pred_std'] ** 2 + data['alt_pred_std'] ** 2) / 2)

if tp == "shared":
    std =  (data['ref_pred'] - data['alt_pred']).std()
elif tp == "single":
    std = np.sqrt(2) * data['pred_std'] / np.sqrt(data.columns.str.startswith("pred_alt").sum())
else:
    raise NotImplementedError()

data['pred_std'] =  np.sqrt((data['ref_pred_std'] ** 2 + data['alt_pred_std'] ** 2) / 2)

print("All data", pearsonr(data['score'], data['ref_pred'] - data['alt_pred']))

significant = abs(data['ref_pred'] - data['alt_pred']) > std 
data['pred_pref'] = (data['ref_pred'] - data['alt_pred']).apply(lambda x: "ref" if x > 0 else "alt")
data['pred_pref'][~significant] = "no"
data['pref_allele'][data['min_fdr'] > 0.05] = "no"

mask = data['min_fdr'] < 0.05

print("Signigicant data", pearsonr(data[mask]['score'], data[mask]['ref_pred'] - data[mask]['alt_pred']))
crosstab = pd.crosstab(index=data['pref_allele'], columns=data['pred_pref'])
crosstab = crosstab.loc[['ref', 'alt', 'no'], ['ref', 'alt', 'no']]
crosstab

All data PearsonRResult(statistic=0.07868945961330985, pvalue=0.0)


/tmp/ipykernel_3579166/3563733882.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['pred_pref'][~significant] = "no"
/tmp/ipykernel_3579166/3563733882.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['pref_allele'][data['min_fdr'] > 0.05] = "no"


Signigicant data PearsonRResult(statistic=0.25821531396716446, pvalue=9.680540476622922e-146)


pred_pref,ref,alt,no
pref_allele,,,
ref,1944,1148,1726
alt,1150,1936,1676
no,83457,83640,109619


In [112]:
eps =1e-10
tp = "single"
data = pd.read_csv("../human_legnet/K562_asb.tsv", sep="\t")
data['ref_pred'] = data.loc[:, data.columns.str.startswith("pred_ref")].mean(axis=1)
data['alt_pred'] = data.loc[:, data.columns.str.startswith("pred_alt")].mean(axis=1)
data['ref_pred_std'] = data.loc[:, data.columns.str.startswith("pred_ref")].std(axis=1)
data['alt_pred_std'] = data.loc[:, data.columns.str.startswith("pred_alt")].std(axis=1)
data['pref_allele'] = data[['es_mean_ref',
       'es_mean_alt']].apply(lambda x: 'ref' if x.es_mean_ref > x.es_mean_alt else "alt", axis=1)

data['score'] = np.log2(data['min_fdr'] + eps) * (data['pref_allele'].apply(lambda x: -1 if x=='ref' else 1))

data['pred_std'] =  np.sqrt((data['ref_pred_std'] ** 2 + data['alt_pred_std'] ** 2) / 2)

if tp == "shared":
    std =  (data['ref_pred'] - data['alt_pred']).std()
elif tp == "single":
    std = np.sqrt(2) * data['pred_std'] / np.sqrt(data.columns.str.startswith("pred_alt").sum())
else:
    raise NotImplementedError()

data['pred_std'] =  np.sqrt((data['ref_pred_std'] ** 2 + data['alt_pred_std'] ** 2) / 2)

print("All data", pearsonr(data['score'], data['ref_pred'] - data['alt_pred']))

significant = abs(data['ref_pred'] - data['alt_pred']) > std 
data['pred_pref'] = (data['ref_pred'] - data['alt_pred']).apply(lambda x: "ref" if x > 0 else "alt")
data['pred_pref'][~significant] = "no"
data['pref_allele'][data['min_fdr'] > 0.05] = "no"

mask = data['min_fdr'] < 0.05

print("Signigicant data", pearsonr(data[mask]['score'], data[mask]['ref_pred'] - data[mask]['alt_pred']))
crosstab = pd.crosstab(index=data['pref_allele'], columns=data['pred_pref'])
crosstab = crosstab.loc[['ref', 'alt', 'no'], ['ref', 'alt', 'no']]
crosstab

All data PearsonRResult(statistic=0.07020284019780694, pvalue=0.0)


/tmp/ipykernel_3579166/2987647316.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['pred_pref'][~significant] = "no"
/tmp/ipykernel_3579166/2987647316.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['pref_allele'][data['min_fdr'] > 0.05] = "no"


Signigicant data PearsonRResult(statistic=0.16583537426025294, pvalue=1.3864612161920364e-168)


pred_pref,ref,alt,no
pref_allele,,,
ref,5325,3796,3278
alt,4816,6172,4081
no,104998,104093,91380


In [87]:
data.columns[0:20]

Index(['chr', 'pos', 'ID', 'ref', 'alt', 'repeat_type', 'n_peak_calls',
       'n_peak_callers', 'mean_BAD', 'mean_SNP_per_segment', 'n_aggregated',
       'total_cover', 'es_mean_ref', 'es_mean_alt', 'fdrp_bh_ref',
       'fdrp_bh_alt', 'min_fdr', 'score', 'cls',
       'pred_ref_test10_val6_0_forw'],
      dtype='object')

In [53]:

y_true = data['pref_allele'] != "no"
y_pred = data['pred_pref'] != "no"

In [54]:
print(classification_report(y_pred=y_pred, y_true=y_true))

              precision    recall  f1-score   support

       False       0.93      0.93      0.93     31259
        True       0.14      0.14      0.14      2429

    accuracy                           0.88     33688
   macro avg       0.54      0.54      0.54     33688
weighted avg       0.88      0.88      0.88     33688



In [154]:
!du -h -l ../human_legnet/*.tsv

772M	../human_legnet/HepG2_asb.tsv
47M	../human_legnet/HepG2_cov_atac.tsv
42M	../human_legnet/HepG2_cov_dnase.tsv
4,2M	../human_legnet/HepG2_cov.tsv
894M	../human_legnet/K562_asb.tsv
26M	../human_legnet/K562_cov_atac.tsv
96M	../human_legnet/K562_cov_dnase.tsv
234M	../human_legnet/K562_cov.tsv
40M	../human_legnet/pred.tsv


In [ ]:
tp = "shared"
data = pd.read_csv("../human_legnet/HepG2_cov_dnase.tsv", sep="\t")
data['ref_pred'] = data.loc[:, data.columns.str.startswith("pred_ref")].mean(axis=1)
data['alt_pred'] = data.loc[:, data.columns.str.startswith("pred_alt")].mean(axis=1)
N =  data.columns.str.startswith("pred_ref").sum()

#data['pred_std'] =  np.sqrt((data['ref_pred_std'] ** 2 + data['alt_pred_std'] ** 2) / 2)

if tp == "shared":
    var_ref = data.loc[:, data.columns.str.startswith("pred_ref")].var(axis=1).mean()
    var_alt = data.loc[:, data.columns.str.startswith("pred_alt")].var(axis=1).mean()
    var = var_ref + var_alt
    std = np.sqrt(var / N)
    se = std * 1.64485
elif tp == "single":
    var_ref = data.loc[:, data.columns.str.startswith("pred_ref")].var(axis=1)
    var_alt = data.loc[:, data.columns.str.startswith("pred_alt")].var(axis=1)
    var = var_ref + var_alt
    std = np.sqrt(var / N)
    se = std * 1.64485
else:
    raise NotImplementedError()


print("All data", pearsonr(data['score'], data['ref_pred'] - data['alt_pred']))

significant = abs(data['ref_pred'] - data['alt_pred']) > se 
data['pred_pref'] = (data['ref_pred'] - data['alt_pred']).apply(lambda x: "ref" if x > 0 else "alt")
data['pred_pref'][~significant] = "no"
data['pref_allele'][data['fdr_comb_pval'] > 0.05] = "no"

mask = data['fdr_comb_pval'] < 0.05

print("Signigicant data", pearsonr(data[mask]['score'], data[mask]['ref_pred'] - data[mask]['alt_pred']))
crosstab = pd.crosstab(index=data['pref_allele'], columns=data['pred_pref'])
crosstab = crosstab.loc[['ref', 'alt', 'no'], ['ref', 'alt', 'no']]
crosstab